In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [57]:
data = pd.read_csv("./data/2015_사업보고서_01_재무상태표_20200515.txt", sep = "\t", encoding = "cp949")
df = data.copy()

In [58]:
print(df.isnull().sum())
# 결측치 처리 필요!
print(df.info())

재무제표종류             0
종목코드               0
회사명                0
시장구분               0
업종                 0
업종명                0
결산월                0
결산기준일              0
보고서종류              0
통화                 0
항목코드               0
항목명                0
당기              8270
전기              8905
전전기             9621
Unnamed: 15    77067
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77067 entries, 0 to 77066
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   재무제표종류       77067 non-null  object 
 1   종목코드         77067 non-null  object 
 2   회사명          77067 non-null  object 
 3   시장구분         77067 non-null  object 
 4   업종           77067 non-null  int64  
 5   업종명          77067 non-null  object 
 6   결산월          77067 non-null  int64  
 7   결산기준일        77067 non-null  object 
 8   보고서종류        77067 non-null  object 
 9   통화           77067 non-null  object 
 10  항목코드         77067 non-null  obje

In [59]:
# 변수 
# 재무제표종류 종목코드 회사명 시장구분 업종 업종명 결산월 결산기준일 
# 보고서종류 통화 항목코드 항목명 당기 전기 전전기 Unnamed: 15
" ".join(df.columns)

'재무제표종류 종목코드 회사명 시장구분 업종 업종명 결산월 결산기준일 보고서종류 통화 항목코드 항목명 당기 전기 전전기 Unnamed: 15'

In [60]:
# 재무제표종류
print(df["재무제표종류"].unique())
print(df["재무제표종류"].value_counts())

['재무상태표, 유동/비유동법-별도재무제표' '재무상태표, 유동성배열법-별도재무제표']
재무상태표, 유동/비유동법-별도재무제표    76661
재무상태표, 유동성배열법-별도재무제표       406
Name: 재무제표종류, dtype: int64


In [61]:
# 종목코드 : 한국거래소에서 관리하는 회사코드로 6자리로 구성
print(len(df["종목코드"].unique()))
print(df["종목코드"].value_counts())

1800
[054050]    210
[036460]    150
[095720]    126
[001790]    125
[052690]    124
           ... 
[208870]     20
[218150]     20
[208370]     20
[222080]     20
[230240]     19
Name: 종목코드, Length: 1800, dtype: int64


In [62]:
# 회사명 : 제출인의 회사명칭
print(len(df["회사명"].unique()))
print(df["회사명"].value_counts())

1801
농우바이오         210
한국가스공사        150
웅진씽크빅         126
대한제당          125
한전기술          124
             ... 
하나머스트4호스팩      20
하나머스트5호스팩      20
한국3호스팩         20
하나머스트2호스팩      20
하나금융7호SPAC     19
Name: 회사명, Length: 1801, dtype: int64


In [63]:
# 시장구분 : 유가증권시장상장법인, 코스닥시장상장법인(Kosdaq)
# 유가증권시장상장법인 : Kospi
# 코스닥 시장은 IT(Information technology), BT(Bio technology), CT(Culture technology) 
# 기업과 벤처기업의 자금조달을 목적으로 1996년 7월 개설된 첨단 벤처기업 중심 시장입니다.
print(df["시장구분"].unique())

['코스닥시장상장법인' '유가증권시장상장법인']


In [64]:
# 업종, 업종명
# 통계청 통계분류 포털 : https://kssc.kostat.go.kr:8443/ksscNew_web/link.do?gubun=001
print(df["업종명"].value_counts())
company_name = df["업종명"].unique()
company_name_sum = df["업종명"].value_counts().sort_values()
df_company_name = pd.DataFrame({"업종명":company_name, "합계":company_name_sum})

# 업종
print(df["업종"].value_counts())
company = df["업종"].unique()
company_sum = df["업종"].value_counts().sort_values()
df_company = pd.DataFrame({"업종":company, "합계":company_sum})

# 업종과 업종명이 다른 행 추출
com_left = pd.merge(df_company_name, df_company, how = "left")
print(com_left[com_left["업종"].isnull()])

com_right = pd.merge(df_company_name, df_company, how = "right")
print(com_right[com_right["업종명"].isnull()])

# 업종 != 업종명 처리방법 회의 필요

전자부품 제조업              4731
특수 목적용 기계 제조업         4151
자동차 부품 제조업            3576
소프트웨어 개발 및 공급업        3254
의약품 제조업               3033
                      ... 
그외 기타 운송장비 제조업          34
소화물 전문 운송업              34
자동차 부품 및 내장품 판매업        33
인형,장난감 및 오락용품 제조업       32
과실, 채소 가공 및 저장 처리업      32
Name: 업종명, Length: 155, dtype: int64
262    4731
292    4151
303    3651
582    3254
212    3033
       ... 
26       35
494      34
452      33
334      32
103      32
Name: 업종, Length: 145, dtype: int64
                                업종명    합계  업종
32               유원지 및 기타 오락관련 서비스업    42 NaN
33                           건물 건설업    42 NaN
79                       소화물 전문 운송업   114 NaN
90                     방적 및 가공사 제조업   123 NaN
91                      1차 비철금속 제조업   124 NaN
143                           어로 어업   367 NaN
152                    그외 기타 제품 제조업   468 NaN
180  전동기, 발전기 및 전기 변환ㆍ 공급ㆍ제어 장치 제조업  1121 NaN
184             건축자재, 철물 및 난방장치 도매업  1671 NaN
189                    기타 상품 전문 소매업  2

In [65]:
# 결산월, 결산기준일
print(df["결산월"].unique())
print(df["결산기준일"].unique())

[ 3 12  9  6  5  8  4 11]
['2015-03-31' '2015-12-31' '2015-09-30' '2015-06-30' '2015-05-31'
 '2015-08-31' '2015-04-30' '2015-10-31' '2015-11-30']


In [66]:
# 보고서종류
print(df["보고서종류"].unique())

['사업보고서']


In [67]:
# 통화 : 필요 없는 변수
# 항목코드 : 계정과목코드
# ifrs_xxx : 국제 기준
# dart_xxx : 국제 기준을 수정
# entity_xxx : 신규회사

In [68]:
df[df["전전기"].isnull()].to_csv("./data/결측치확인.csv", index = False,encoding = 'utf-8')
no = df[df["전전기"].isnull()]
no_lst = no["항목명"].unique()
no_big_classifier = []
for i in no_lst:
    if "[" not in i:
        no_big_classifier.append(i)

In [69]:
# no_big_classifier : 

In [70]:
# data
df["당기"] = df["당기"].str.replace(",", "")
df["전기"] = df["전기"].str.replace(",", "")
df["전전기"] = df["전전기"].str.replace(",", "")
df[["당기", "전기", "전전기"]] = df[["당기", "전기", "전전기"]].astype(np.float32)

In [71]:
# 필요 없는 변수 제거
df.drop("통화", axis = 1, inplace = True)
df.drop("종목코드", axis = 1, inplace = True)
df.drop("Unnamed: 15", axis = 1, inplace = True)

In [72]:
sort = list(df["항목명"].unique())

dic = {i:["-"]*1801 for i in sort}

In [73]:
a = df[df["회사명"] == "3S"][["항목명", "당기"]].T
a.rename(columns = a.iloc[0,:], inplace = True)
a.drop(index = "항목명", inplace = True)

In [ ]:
a
idx = 0
for i in range(len(a.columns)):
    dic[a.columns[i]][0] = a.iloc[0, i]

In [ ]:
lst = []
for i in sort:
    if dic[i][0] != "-":
        lst.append(i)

In [ ]:
lst.append(0)
lst.append(0)
pd.DataFrame({"3S":a.columns, "lst":lst})
# 금융리스부채

In [ ]:
a.drop("회사명", axis = 1, inplace=True)
a.reset_index(drop = True, inplace = True)

In [ ]:
b = pd.DataFrame({"회사명":["3S"]})

In [ ]:
pd.concat([b, a], axis = 1).to_csv("./data/3S전치행렬.csv", encoding = "euc-kr", index = False)

In [ ]:
# 회사명 
company = list(df["회사명"].unique())

col_num = []
col_num_name = []
for idx, name in enumerate(company):
    a = df[df["회사명"] == name][["항목명", "당기"]].T
    a.rename(columns = a.iloc[0,:], inplace = True)
    a.drop(index = "항목명", inplace = True)
    col_num.append(len(a.columns))
    col_num_name.append(name)
    
# 가장 많은 항목명을 가진 회사
for i in range(len(col_num)):
    if col_num[i] == max(col_num):
        print(col_num_name[i])

In [ ]:
a = df[df["회사명"] == "농우바이오"][["항목명", "당기"]].T
a.rename(columns = a.iloc[0,:], inplace = True)
a.drop(index = "항목명", inplace = True)
print(a)

In [ ]:
# '      금융리스부채'
# '      금융리스부채'(25,26, 34, 35)
df[df["회사명"] == "3S"].duplicated(["항목명"], keep = False)
df[df["회사명"] == "3S"]["항목명"][[25,26, 34, 35]]

In [ ]:
a = pd.DataFrame({"a":[1, 3, 4, 5, 1]})
a.duplicated("a", keep = False)

In [ ]:
df["항목명"].unique()

In [ ]:
df["항목명"][0]

In [ ]:
lst1_대분류 = []
for i in range(len(df["항목명"])):
    if df["항목명"][i][0] != " " and df["당기"].isnull()[i] == True:
        lst1_대분류.append(df["항목명"][i])
        
print(set(lst1_대분류))
pd.DataFrame({"대분류":lst1_대분류})["대분류"].unique()

In [ ]:
lst_중분류 = []
for i in range(len(df["항목명"])):
    if df["항목명"][i][:3] == "   " and df["당기"].isnull()[i] == True:
        lst_중분류.append(df["항목명"][i])
        
print(set(lst1))
pd.DataFrame({"대분류":lst_중분류})["대분류"].unique()

In [ ]:
df[["항목코드", "항목명"]].iloc[:40, :]

In [ ]:
# 항목코드 중복 확인
lst_entity_코드 = []
lst_entity_명 = []
lst_without_entity_코드 = []
lst_without_entity_명 = []

# 각 리스트에 코드 및 코드명 append
for idx in range(len(df)):
    if "entity" in df["항목코드"][idx]:
        lst_entity_코드.append(df["항목코드"][idx])
        lst_entity_명.append(df["항목명"][idx])
    else:
        lst_without_entity_코드.append(df["항목코드"][idx])
        lst_without_entity_명.append(df["항목명"][idx])
        
# !entity 코드 딕셔너리
dic_without_entity = {lst_without_entity_코드[idx]:[] for idx in range(len(lst_without_entity_코드))}

for i, j in zip(lst_without_entity_코드, lst_without_entity_명):
    dic_without_entity[i].append(j)
    
check_duplication = {i:len(set(j)) for i, j in dic_without_entity.items()}
check_duplication

In [ ]:
dic_without_entity

In [ ]:
for idx in range(len(df)):
    if "NoncurrentAssetsOrDisposalGroupsClassifiedAsHeldForSaleOrAsHeldForDistributionToOwners" in df["항목코드"][idx]:
        print("항목코드 =", df["항목코드"][idx], "항목명 =", df["항목명"][idx], "idx =", idx)

In [ ]:
dic_entity_코드_명 = {lst_entity_코드[idx]:[] for idx in range(len(lst_entity_코드))}

for i, j in zip(lst_entity_코드, lst_entity_명):
    dic_entity[i].append(j)
    
dic_entity_코드_명

In [ ]:
dic_entity_명_코드 = {lst_entity_명[idx]:[] for idx in range(len(lst_entity_명))}

for i, j in zip(lst_entity_명, lst_entity_코드):
    dic_entity_명_코드[i].append(j)
    
dic_entity_명_코드

In [54]:
elementid = pd.read_excel("./data/재무제표양식.xlsx", encoding = "utf-8", sheet_name="BS1")
# '한글 Label', 'Element ID'
df_element = elementid[['한글 Label', 'Element ID']]
df_element = df_element[df_element["Element ID"].notnull()]

df_element.columns = df_element.columns.str.replace(" ", "_")
lst_element = df_element["Element_ID"].unique()

re_lst_element = []
for i in lst_element:
    a = re.sub("[a-z]{1,}-[a-z]{1,}", "", i)
    b = re.sub("[dart]", "", a)
    re_lst_element.append(b)

dic_element = {i:[] for i in re_lst_element}
for i, j in zip(re_lst_element, df_element["한글_Label"]):
    dic_element[i].append(j)

In [90]:
print(df[["항목코드", "항목명"]].iloc[0,:])
print(dic_element)

# dic_element[re.findall("_\w{1,}", df["항목코드"][0])[0]]
# for idx in range(len(df)):
#     if "entity" in df["항목코드"][idx]:
#         lst_entity_코드.append(df["항목코드"][idx])
#         lst_entity_명.append(df["항목명"][idx])
#     else:
#         lst_without_entity_코드.append(df["항목코드"][idx])
#         lst_without_entity_명.append(df["항목명"][idx])

항목코드    ifrs_AssetsAbstract
항목명           자산 [abstract]
Name: 0, dtype: object
{'_SemenOfFinncilPosiionAbsc': ['재무상태표 [abstract]'], '_AssesAbsc': ['자산 [abstract]'], '_CuenAsses': ['유동자산'], '_CshAnCshEquivlens': ['현금및현금성자산'], '_Csh': ['현금'], '_CshEquivlens': ['현금성자산'], '_GovenmenGnsCshAnCshEquivlensGoss': ['정부보조금'], '_ShoTemDeposisNoClssifieAsCshEquivlens': ['단기금융상품'], '_TeAnOheCuenReceivbles': ['매출채권 및 기타유동채권'], '_ShoTemTeReceivble': ['매출채권'], '_AllownceFoDoubfulAcccounShoTemTeReceivble': ['대손충당금'], '_CuenFinnceLeseReceivbles': ['단기금융리스채권'], '_AllownceFoDoubfulAcccounCuenFinnceLeseReceivbles': ['대손충당금'], '_ShoTemDefeeAncillyIncomeFoLons': ['단기이연대출부대수익'], '_AllownceFoDoubfulAcccounShoTemDefeeAncillyIncomeFoLons': ['대손충당금'], '_ShoTemOheReceivbles': ['단기미수금'], '_AllownceFoDoubfulAcccounShoTemOheReceivbles': ['대손충당금'], '_ReceivblesOnConsucionConcs': ['공사미수금'], '_AllownceFoDoubfulAcccounReceivblesOnConsucionConcs': ['대손충당금'], '_ReceivblesReleseSles': ['분양미수금'], '_AllownceFoDoubfulAcccounRec